# Exploring the SPC tornado database

Let's explore the SPC tornado database. Data may be downloaded from: https://www.spc.noaa.gov/wcm/data/1950-2018_actual_tornadoes.csv

In [ ]:
#set up environment
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display
import numpy as np
import datetime
import pandas as pd
import seaborn as sns
import pytz
import warnings

warnings.simplefilter("ignore")
sns.set_style('darkgrid')
%matplotlib inline

Using Pandas, we can read in csv files very easily! Further, we can read the data in as a DataFrame, a type of 2-D labeled data structure in Pandas that is akin to a table or spreadsheet. There are a lot of nifty ways to analyze and visualize Pandas dataframes, so let'd dive into the data!

In [ ]:
#df = pd.read_csv('https://www.spc.noaa.gov/wcm/data/1950-2018_actual_tornadoes.csv')
df = pd.read_csv('1950-2018_actual_tornadoes.csv')
print('ORIGINAL columns',df.columns)
df.rename(columns={'mag': 'tor_rate'},inplace=True)
print(df.columns)
#we can drop unnecessary columns, such as 'om' for example
df.drop('om',axis=1,inplace=True)
print(df.columns)

In [ ]:
#tornadoes without a rating are given a value of -9
#estimated F/EF ratings are given by the 'fc' column
#let's replace all values of -9 with the estimate values in fc
print('original unique ratings',np.unique(df['tor_rate']))


In [ ]:
#use numpy where to find the indices where 'tor_rate' ==-9
where = np.where(df['tor_rate']==-9.)[0]
print('where',where)
#use these indices to replace values
df['tor_rate'].iloc[where] = df['fc'].iloc[where]
print(df['tor_rate'])

print('unique ratings',np.unique(df['tor_rate']))

In [ ]:
#create a column of datetimes using 'date' and 'time' columns
df['datetime'] = pd.to_datetime(df['date']+' '+df['time'],format='%Y%m%d %H:%M:%S')
tzs = df['tz']

def convert2UTC(row):
    #this function will convert timezone-aware datetimes to UTC
    if row['tz'] == 3:
        dat = row['datetime'].tz_localize(pytz.timezone('US/Central'))
        datutc = dat.tz_convert(pytz.timezone('UTC'))
        
    elif row['tz'] == 9:
        dat = row['datetime'].tz_localize(pytz.timezone('GMT'))
        datutc = dat.tz_convert(pytz.timezone('UTC'))
        
    else:
        datutc = row['datetime'].tz_localize(pytz.timezone('UTC'))
        
    return datutc

#create a UTC datetime column
df['dates_utc'] = df.apply(lambda row: convert2UTC(row), axis=1)
#dates_utc should now be a new column in the dataframe
print(df['dates_utc'],df['dates_utc'].values)
#we often will want to aggregate tornadoes by convective day (12-12Z)
df['convective_day'] = df['dates_utc'].dt.strftime('%Y-%m-%d')
#we can set a column to have the name of the month
df['month_name']  = df['dates_utc'].dt.strftime('%B')



Let's start exploring our data and making some plots! First, let's create a timeseries of annual tornado counts by rating

In [ ]:
#use 'groupby' to aggregate by year and rating
group = df.groupby(['yr','tor_rate']).agg(tornado_count = pd.NamedAgg('tor_rate','count'))
print(group)

#print('index',group.index,'names of each level of index',group.index.names)

In [ ]:
fig = plt.figure()
fig.set_size_inches(8,6)
sns.lineplot(x='yr',y='tornado_count',hue='tor_rate',data=group,palette=sns.color_palette("hls", 6))
#now let's plot a line with all F/EF-1+ tornadoes
sns.lineplot(x='yr',y='tornado_count',data=group.drop(0, level='tor_rate').groupby(level=[0]).sum(),color='k')
plt.ylim(ymin=0)
plt.xlim(group.index.get_level_values('yr').min(),group.index.get_level_values('yr').max())
plt.show()

We can also compare tornado counts vs. tornado days

In [ ]:
df['tor_day'] = df['dates_utc'].dt.strftime('%Y-%m-%d')
torday_group = df.groupby(['yr','tor_rate','tor_day']).agg(tornado_day = pd.NamedAgg('tor_day','nunique')).groupby(['yr','tor_rate']).sum()
print(torday_group)

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=1,figsize=(8,6))
twin_ax = ax.twinx()
g1 = sns.lineplot(x='yr',y='tornado_count',hue='tor_rate',data=group,palette=sns.color_palette("hls", 6),ax=ax)
g2 = sns.lineplot(x='yr',y='tornado_day',hue='tor_rate',data=torday_group,palette=sns.color_palette("hls", 6),linestyle='--',ax=twin_ax)
ax.set_ylim(ymin=0)
twin_ax.set_ylim(ymin=0)
ax.grid(False)
twin_ax.grid(False)
g1.legend_.remove()
plt.xlim(group.index.get_level_values('yr').min(),group.index.get_level_values('yr').max())
plt.show()

Are tornado days and tornado counts correlated?

In [ ]:
#fill in missing groups with zero
group = group.unstack(fill_value=0).stack()
torday_group = torday_group.unstack(fill_value=0).stack()
for irx,rate in enumerate(np.arange(6)):
    #extract time series from the groupby dfs by rating
    #xs allows us to select multiindex values
    tcount = group.xs(rate,level=1)
    tday = torday_group.xs(rate,level=1)
    pearsonr = tcount.corrwith(other=tday['tornado_day'],method='pearson',axis=0)
    print('rating:',rate)
    print('----------------')
    print('pearsonr:',pearsonr)
    print()

Now, let's look at the distribution of tornadoes by F/EF rating 

In [ ]:
fig = plt.figure()
fig.set_size_inches(8,6)
#first, we need reset our index from a multiindex
annual_sum = group.reset_index()
#plot boxplots by rating using Seaborn
sns.boxplot(x='tor_rate',y='tornado_count',data=annual_sum,palette=sns.color_palette("hls", 6))
plt.show()

In [ ]:
df2 =df.drop(df[df['tor_rate'] == 0].index)
mongroup = df2.groupby(['yr','mo','tor_rate']).agg(tornado_count = pd.NamedAgg('tor_rate','count'))
#fill in missing groups with zero
mongroup = mongroup.unstack(fill_value=0).stack()
monthnames = pd.unique(df['month_name'])
idx = pd.IndexSlice
fig,ax = plt.subplots(nrows=3,ncols=4,figsize=(20,18))
for iax,axs in enumerate(ax.ravel()):
    #sns.lineplot(x='yr',y='tornado_count',hue='tor_rate',data=mongroup.loc[idx[:,iax+1],:],ax=axs,palette=sns.color_palette("hls", 5))
    sns.lineplot(x='yr',y='tornado_count',data=mongroup.loc[idx[:,iax+1],:].groupby(level=[0]).sum(),ax=axs,color='k')
    axs.set_title('{}'.format(monthnames[iax]),fontsize='large',weight='bold')
    axs.set_ylim(ymin=0)
    axs.set_xlim(mongroup.index.get_level_values('yr').min(),mongroup.index.get_level_values('yr').max())
plt.show()

In [ ]:
print('group',group)
ef0plus = group.groupby(level=[0]).sum()
ef1plus = group.drop(0, level='tor_rate').groupby(level=[0]).sum()
ef2plus = group.drop(1, level='tor_rate').groupby(level=[0]).sum()

dfs = [ef0plus,ef1plus,ef2plus]
colors = ['k','r','b']
fig,ax = plt.subplots(ncols=1,nrows=len(dfs),figsize=(10,14))
for idx,dff in enumerate(dfs):
    #find the year-to-year difference (first-order difference)
    diff = dff.diff(periods=1,axis = 0).fillna(0)
    sns.lineplot(x='yr',y='tornado_count',data=diff,color=colors[idx],ax=ax.ravel()[idx])
    ax[idx].set_xlim(diff.index.get_level_values('yr').min(),diff.index.get_level_values('yr').max())
    ax[idx].set_title('First-Order Difference of Annual F/EF-{}+ Tornadoes'.format(idx))
plt.show()

In [ ]:
import ruptures as rpt
fig,ax = plt.subplots(ncols=1,nrows=len(dfs),figsize=(10,14))
model='l2' #piecewise linear model
years = dff.index.values
print('year',years)
for idx,dff in enumerate(dfs):
    #find the year-to-year difference (first-order difference)
    diff = dff.diff(periods=1,axis = 0).fillna(0)
    print(diff)
    sns.lineplot(x='yr',y='tornado_count',data=diff,color=colors[idx],ax=ax.ravel()[idx])
    ax[idx].set_xlim(diff.index.get_level_values('yr').min(),diff.index.get_level_values('yr').max())
    algo = rpt.Pelt(model=model,min_size=15).fit(diff['tornado_count'].to_numpy().squeeze())
    chpts = np.array(algo.predict(pen=3))[0:-1]
    for c in chpts:
        ax[idx].axvline(years[c],color=colors[idx],linestyle=':',linewidth=3)
    ax[idx].set_title('First-Order Difference of Annual F/EF-{}+ Tornadoes'.format(idx))
plt.show()

# Plot tornado tracks

In [ ]:
apr27 = df.loc[df.convective_day=='2011-04-27']
print(apr27.columns)
print('Total fatalities:',apr27['fat'].sum())
print('Total injuries:',apr27['inj'].sum())
print('Total tornado pathlength (mi):',apr27['len'].sum())
apr27group = apr27.groupby(['tor_rate']).agg(numberOfTors=pd.NamedAgg('tor_rate','count'),fatalities=pd.NamedAgg('fat','sum'),injuries=pd.NamedAgg('inj','sum'),total_pathlength=pd.NamedAgg('len','sum'))
print(apr27group)


In [ ]:
import matplotlib.path as Path
import matplotlib.patches as patches
import os
os.environ['PROJ_LIB'] = r'/home/khoogewi/.conda/envs/cent7/5.3.1-py37/py37/share/proj'
from mpl_toolkits.basemap import Basemap
import matplotlib.lines as mlines

minlat = 26
maxlat = 41 
minlon = -98
maxlon = -72
lat_0 = 60.
lon_0 = 262.-360.
latin1 = 60.
latin2 = 30.


fig,ax = plt.subplots(nrows=1,ncols=1,figsize=(12,8))
m = Basemap(projection='lcc',lat_0=lat_0,lon_0=lon_0,
       llcrnrlat=minlat,urcrnrlat=maxlat, llcrnrlon=minlon,urcrnrlon=maxlon,
       lat_1=latin1,lat_2=latin2,resolution='h',area_thresh=10000.)
#m.shadedrelief(alpha=0.25)
m.drawlsmask(ocean_color='azure',land_color='wheat')
m.drawcounties(linewidth=0.15,color='grey')
m.drawcoastlines(linewidth=1.)
m.drawstates(linewidth=0.75)
m.drawcountries(linewidth=1.)
parallels = np.arange(0.,90,5.)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10,color='dimgrey')
meridians = np.arange(180.,360.,10.)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10,color='dimgrey')


trate,btorlat,btorlon,etorlat,etorlon = apr27['tor_rate'].values,apr27['slat'].values,apr27['slon'].values,apr27['elat'].values,apr27['elon'].values

for rate,blat,blon,elat,elon in list(zip(trate,btorlat,btorlon,etorlat,etorlon)):
    if rate== 0:
       color='purple'
    elif rate == 1:
       color='blue'
    elif rate == 2:
       color='green'
    elif rate == 3:
       color='magenta'
    elif rate == 4:
       color='orange'
    elif rate == 5:
       color = 'red'
    else:
       color = 'black'
    #some reports do not have an ending lat/lon of tor path
    if elon == 0. or elat == 0.:
       x1,y1 = m(blon,blat)
       x2,y2 = m(blon,blat)
       ax.plot(x1,y1,marker='o',markersize=1,color=color)
    else:
       x1,y1 = m(blon,blat)
       x2,y2 = m(elon,elat)
       #ax.plot(x1,y1,marker='o',markersize=0.75,color=color)  
    cpath = matplotlib.path.Path(((x1,y1),(x2,y2)))
    cpatch=patches.PathPatch(cpath,edgecolor=color,lw=2)
    ax.add_patch(cpatch)
    
colors = ['purple','blue','green','magenta','orange','red']
ratings = np.arange(0,6,1)
lines = []
for color,rate2 in zip(colors[rate:],ratings[rate:]):
        lines.append(mlines.Line2D([], [], color=color,
                          markersize=15, label='EF-%s'%rate2))# for color,rate in zip(colors,ratings)]
plt.title('April 27, 2011 Tornadoes',fontsize='xx-large',weight='bold')
labels = ["EF-%s"%rat for rat in np.arange(1,6,1)]
fig.legend(lines,labels,loc='best',title='Tornado Rating')
plt.show()

    
